# Modelo de Análisis de Sentimiento

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset # Import the Dataset class
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer # Import Trainer

In [ ]:
df=pd.read_csv('/content/tweets_con_sentimiento.csv', engine='python')

In [ ]:
df['sentiment'].value_counts()

In [ ]:
import re

def limpiar_tweet(texto):
    texto = texto.lower()
    texto = re.sub(r"http\S+|www\S+|https\S+", "", texto)  # URLs
    texto = re.sub(r"\@\w+|\#", "", texto)  # menciones y hashtags
    texto = re.sub(r"\s+", " ", texto).strip()  # espacios múltiples
    return texto

df["text"] = df["txt"].apply(limpiar_tweet)

In [ ]:
df_train=df[['text', 'sentiment']].copy()

In [ ]:
# 4. MAPEAR ETIQUETAS
label_list = df_train["sentiment"].unique().tolist()  # ['neutral', 'negative', 'positive']
label_to_id = {label: idx for idx, label in enumerate(label_list)}
id_to_label = {v: k for k, v in label_to_id.items()}

df_train["labels"] = df_train["sentiment"].map(label_to_id)

# Cargar CSV

# Separar 70/30
train_df, val_df = train_test_split(df_train, test_size=0.3, random_state=42, stratify=df_train['labels'])

# Convertir a Dataset de HuggingFace
train_dataset = Dataset.from_pandas(train_df[["text", "labels"]])
val_dataset = Dataset.from_pandas(val_df[["text", "labels"]])

In [ ]:

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

train_tokenized = train_dataset.map(tokenize_function, batched=True)
val_tokenized = val_dataset.map(tokenize_function, batched=True)


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "roberta-base",
    num_labels=len(label_list),
    id2label=id_to_label,
    label2id=label_to_id
)
for param in model.parameters():
    param.requires_grad = False

num_encoder_layers = len(model.roberta.encoder.layer)
start_unfreeze_idx = max(0, num_encoder_layers - 3)
for i in range(start_unfreeze_idx, num_encoder_layers):
    for param in model.roberta.encoder.layer[i].parameters():
        param.requires_grad = True

# Unfreeze the classification head.
for param in model.classifier.parameters():
    param.requires_grad = True

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=1,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to=[]   #
)

# Métricas personalizadas
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
from sklearn.metrics import classification_report

predictions = trainer.predict(val_tokenized)
y_pred = np.argmax(predictions.predictions, axis=1)
y_true = predictions.label_ids

print(classification_report(y_true, y_pred, target_names=label_list))

In [ ]:
trainer.save_model("./modelo_sentimientos")

from google.colab import drive
drive.mount('/content/drive')
output_dir = "/content/drive/MyDrive/mi_modelo_sentimientos"

trainer.save_model(output_dir)

print(f"Modelo y tokenizador guardados en: {output_dir}")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Guarda el modelo entrenado
trainer.save_model("./modelo")

# Guarda el tokenizador (opcional, pero recomendado)
tokenizer.save_pretrained("./tokenizador")

In [ ]:
from google.colab import files
files.download('/content/modelo_sentimientos.zip')